In [1]:
#
# Below needed for GRU
#
import pandas as pd
import datetime
import numpy as np
import gc

import utils
utils.widen_ipython_window()


In [9]:
# Paths

MAIN_PATH = "/home/mahesh/Desktop/ML/kaggle/amex/"

# Data
PATH_TO_DATA                = MAIN_PATH + "data/"
PATH_TO_PROCESSED2_DATA     = PATH_TO_DATA + "processed2/"

FILENAME_GRU_SUBMISSION     = PATH_TO_PROCESSED2_DATA + "submission_gru.csv"
FILENAME_LGBM_SUBMISSION    = PATH_TO_PROCESSED2_DATA + "submission_lgbm.csv"

FILENAME_GRU_LGBM_SUBMISSION    = PATH_TO_PROCESSED2_DATA + "submission_gru_lgbm.csv"

FILENAME_GRU_LGBM_MIN_MAX_SUBMISSION    = PATH_TO_PROCESSED2_DATA + "submission_gru_lgbm_min_max.csv"
FILENAME_GRU_LGBM_HIGH_CONFIDENCE_SUBMISSION    = PATH_TO_PROCESSED2_DATA + "submission_gru_lgbm_high_confidence.csv"

In [6]:
gru_preds  = pd.read_csv(FILENAME_GRU_SUBMISSION)
lgbm_preds = pd.read_csv(FILENAME_LGBM_SUBMISSION)

gru_preds.sort_values(by=['customer_ID'],inplace=True)
lgbm_preds.sort_values(by=['customer_ID'],inplace=True)

avg_preds = (gru_preds.prediction.values + lgbm_preds.prediction.values)/2

avg_preds_df = pd.DataFrame({'customer_ID':gru_preds['customer_ID'].values, 'prediction':avg_preds})

avg_preds_df.to_csv(FILENAME_GRU_LGBM_SUBMISSION, index=False)

In [8]:
gru_preds  = pd.read_csv(FILENAME_GRU_SUBMISSION)
lgbm_preds = pd.read_csv(FILENAME_LGBM_SUBMISSION)

gru_preds.sort_values(by=['customer_ID'],inplace=True)
lgbm_preds.sort_values(by=['customer_ID'],inplace=True)

avg_preds = (gru_preds.prediction.values + lgbm_preds.prediction.values)/2

max_preds = np.maximum(gru_preds.prediction.values, lgbm_preds.prediction.values)

min_preds = np.minimum(gru_preds.prediction.values, lgbm_preds.prediction.values)

final_preds = np.where((gru_preds.prediction.values + lgbm_preds.prediction.values)/2 >= 0.5, max_preds, min_preds)


final_preds_df = pd.DataFrame({'customer_ID':gru_preds['customer_ID'].values, 'prediction':final_preds})

final_preds_df.to_csv(FILENAME_GRU_LGBM_MIN_MAX_SUBMISSION, index=False)

In [12]:
gru_preds_pd  = pd.read_csv(FILENAME_GRU_SUBMISSION)
lgbm_preds_pd = pd.read_csv(FILENAME_LGBM_SUBMISSION)

gru_preds_pd.sort_values(by=['customer_ID'],inplace=True)
lgbm_preds_pd.sort_values(by=['customer_ID'],inplace=True)

gru_preds = gru_preds_pd.prediction.values
lgbm_preds = lgbm_preds_pd.prediction.values

# both_above_th = np.bitwise_and((gru_preds>=0.5),(lgbm_preds>=0.5))
# both_below_th = np.bitwise_and((gru_preds<0.5) ,(lgbm_preds<0.5))

# max_preds = np.maximum(gru_preds, lgbm_preds)
# min_preds = np.minimum(gru_preds, lgbm_preds)

gru_confidence = np.where(gru_preds >= 0.5, (gru_preds-0.5), (0.5-gru_preds))
lgbm_confidence = np.where(lgbm_preds >= 0.5, (lgbm_preds-0.5), (0.5-lgbm_preds))

high_confidence_pred = np.where(gru_confidence > lgbm_confidence, gru_preds, lgbm_preds)

final_preds_df = pd.DataFrame({'customer_ID':gru_preds_pd['customer_ID'].values, 'prediction':high_confidence_pred})

final_preds_df.to_csv(FILENAME_GRU_LGBM_HIGH_CONFIDENCE_SUBMISSION, index=False)

In [13]:
u_preds_pd  = pd.read_csv(FILENAME_GRU_SUBMISSION)
lgbm_preds_pd = pd.read_csv(FILENAME_LGBM_SUBMISSION)

gru_preds_pd.sort_values(by=['customer_ID'],inplace=True)
lgbm_preds_pd.sort_values(by=['customer_ID'],inplace=True)

gru_preds = gru_preds_pd.prediction.values
lgbm_preds = lgbm_preds_pd.prediction.values

both_above_th = np.bitwise_and((gru_preds>=0.5),(lgbm_preds>=0.5))
both_below_th = np.bitwise_and((gru_preds<0.5) ,(lgbm_preds<0.5))

# max_preds = np.maximum(gru_preds, lgbm_preds)
# min_preds = np.minimum(gru_preds, lgbm_preds)

gru_confidence = np.where(gru_preds >= 0.5, (gru_preds-0.5), (0.5-gru_preds))
lgbm_confidence = np.where(lgbm_preds >= 0.5, (lgbm_preds-0.5), (0.5-lgbm_preds))

high_confidence_pred = np.where(gru_confidence > lgbm_confidence, gru_preds, lgbm_preds)

# final_preds_df = pd.DataFrame({'customer_ID':gru_preds_pd['customer_ID'].values, 'prediction':high_confidence_pred})

# final_preds_df.to_csv(FILENAME_GRU_LGBM_HIGH_CONFIDENCE_SUBMISSION, index=False)

In [16]:
c1 = np.count_nonzero(~(np.bitwise_or(both_above_th, both_below_th) ))
print(c1)
c2 = np.count_nonzero((np.bitwise_or(both_above_th, both_below_th) ))
print(c2)

26549
898072
